In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [161]:
first_flag = True
df_all = None
for folder in os.listdir('football_data/'):
    if ('.' in folder):
        continue
    for csv in os.listdir('football_data/{}/'.format(folder)):
        if csv.endswith(".csv"):
            try:
                df_part = pd.read_csv('football_data/{}/{}'.format(folder, csv), error_bad_lines=False, warn_bad_lines=False)
            except Exception as e:
                print('football_data/{}/{}'.format(folder, csv))


            if 'Div' in (df_part.columns): #format1
                df_part['Country'] = folder
            else:
                df_part.rename(columns={'League': 'Div',
                                       'Home': 'HomeTeam',
                                       'Away': 'AwayTeam',
                                       'HG': 'FTHG',
                                       'AG': 'FTAG'}, inplace=True)
            df_part = df_part.loc[:, ['Country', 'Div', 'Date', 'HomeTeam',
                               'AwayTeam','FTHG','FTAG']]
            if first_flag:
                first_flag = False
                df_all = df_part
            else:
                df_all = df_all.append(df_part, ignore_index=True)

/home/liaojiping/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [162]:
df_all.dropna(inplace=True)
df_all.reset_index(inplace=True, drop=True)
df_all.tail()

,Country,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG
204906,England,E1,06/05/07,Preston,Birmingham,1.0,0.0
204907,England,E1,06/05/07,QPR,Stoke,1.0,1.0
204908,England,E1,06/05/07,Sheffield Weds,Norwich,3.0,2.0
204909,England,E1,06/05/07,Southampton,Southend,4.0,1.0
204910,England,E1,06/05/07,West Brom,Barnsley,7.0,0.0


In [163]:
df_all['Date'] = df_all['Date'].astype(str)
df_all['Country'] = df_all['Country'].str.strip()
df_all['Div'] = df_all['Div'].str.strip()

In [164]:
df_all.sort_values(['Country', 'Div', 'Date'], inplace=True)

In [165]:
df_all = df_all[['Country', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG']]

df_all['total_goals'] = df_all['FTHG'] + df_all['FTAG']

df_all['total_goals_odd'] = df_all['total_goals'] % 2

# df_all['total_goals_over2'] = df_all['total_goals'] > 2

In [166]:
def year_fn(x):
    if len(x) > 2:
        return x
    elif int(x) > 30:
        return '19' + x
    else:
        return '20' + x

df_all['year'] = df_all['Date'].map(lambda x: x.split('/')[-1][-2:])
df_all['year'] = df_all['year'].map(lambda x: year_fn(x))
df_all['month'] = df_all['Date'].map(lambda x: x.split('/')[1])
#match season
df_all.loc[df_all['month']>'06', 'year'] = df_all.loc[df_all['month']>'06', 'year'].map(lambda x: str(int(x)+1))

In [167]:
df_all.groupby('Country').count()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,total_goals,total_goals_odd,year,month
Country,,,,,,,,,,
Argentina,2679,2679,2679,2679,2679,2679,2679,2679,2679,2679
Austria,1190,1190,1190,1190,1190,1190,1190,1190,1190,1190
Belgium,6235,6235,6235,6235,6235,6235,6235,6235,6235,6235
Brazil,2659,2659,2659,2659,2659,2659,2659,2659,2659,2659
China,1200,1200,1200,1200,1200,1200,1200,1200,1200,1200
Denmark,1442,1442,1442,1442,1442,1442,1442,1442,1442,1442
England,57814,57814,57814,57814,57814,57814,57814,57814,57814,57814
Finland,1394,1394,1394,1394,1394,1394,1394,1394,1394,1394
France,17028,17028,17028,17028,17028,17028,17028,17028,17028,17028


In [168]:
df_all_raw = df_all.copy()
df_all_raw = df_all_raw[['Country', 'Div', 'year', 'month','total_goals_odd']]
df_all_raw = df_all_raw[df_all_raw['year'] >='2000']
df_all_raw['Country_Div'] = df_all_raw['Country'] + '_' +df_all_raw['Div']

In [169]:
df_all_raw['year'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2004',
       '2009', '2000', '2008', '2001', '2002', '2006', '2007', '2005',
       '2010', '2011', '2003', '2012'], dtype=object)

In [170]:
df_all_raw['Country_Div'].value_counts()

England_E1                    10540
England_E3                    10350
England_E2                    10336
Spain_SP2                      8365
Italy_I2                       7831
England_EC                     7334
Spain_SP1                      7277
England_E0                     7246
France_F1                      6831
Italy_I1                       6828
France_F2                      6761
Germany_D1                     5839
Turkey_T1                      5739
Netherlands_N1                 5679
Germany_D2                     5487
Portugal_P1                    5156
Belgium_B1                     5011
Scotland_SC0                   4126
Greece_G1                      3669
Scotland_SC2                   3408
Scotland_SC1                   3392
Scotland_SC3                   3316
Argentina_Primera Division     2679
Brazil_Serie A                 2659
USA_MLS                        2527
Mexico_Liga MX                 2169
Japan_J-League                 2151
Romania_Liga 1              

# Get candidates

In [171]:
for start_year in ['2000', '2005', '2010', '2015']:
    df_all = df_all_raw[df_all_raw['year'] >= start_year]
    df_all_cnt = df_all.groupby(['Country_Div']).count().reset_index()
    df_all_cnt.rename(columns={'year': 'Count'}, inplace=True)
    df_all_cnt = df_all_cnt[df_all_cnt['Count'] > 100]
    df_all_mean = df_all.groupby(['Country_Div']).mean().reset_index()

    df_res = pd.merge(df_all_mean, df_all_cnt[['Country_Div', 'Count']])
    df_res['chance'] = (0.5 - df_res['total_goals_odd']).abs()
    df_res = df_res.sort_values('chance', ascending=False).reset_index(drop=True)
    exec("candidate_{}=df_res[['Country_Div', 'total_goals_odd', 'chance']].head(15)".format(start_year))

In [172]:
candidate_2000.columns = ['Country_Div_2000', 'total_goals_odd_2000', 'chance_2000']
candidate_2005.columns = ['Country_Div_2005', 'total_goals_odd_2005', 'chance_2005']
candidate_2010.columns = ['Country_Div_2010', 'total_goals_odd_2010', 'chance_2010']
candidate_2015.columns = ['Country_Div_2015', 'total_goals_odd_2015', 'chance_2015']

In [173]:
df_candidates = pd.concat([candidate_2000, 
                candidate_2005, 
                candidate_2010,
                candidate_2015], axis=1)

df_candidates

,Country_Div_2000,total_goals_odd_2000,chance_2000,Country_Div_2005,total_goals_odd_2005,chance_2005,Country_Div_2010,total_goals_odd_2010,chance_2010,Country_Div_2015,total_goals_odd_2015,chance_2015
0,Japan_J-League,0.539749,0.039749,Japan_J-League,0.539749,0.039749,Netherlands_N1,0.457516,0.042484,Japan_J-League,0.540636,0.040636
1,Netherlands_N1,0.466103,0.033897,Germany_D2,0.464747,0.035253,Japan_J-League,0.539749,0.039749,Netherlands_N1,0.461874,0.038126
2,Germany_D2,0.467286,0.032714,Netherlands_N1,0.465434,0.034566,Germany_D2,0.463649,0.036351,Germany_D1,0.461874,0.038126
3,France_F2,0.469457,0.030543,France_F1,0.467721,0.032279,France_F2,0.470344,0.029656,Germany_D2,0.463203,0.036797
4,Italy_I2,0.471715,0.028285,Italy_I2,0.472649,0.027351,Italy_I2,0.471462,0.028538,Italy_I2,0.469500,0.030500
5,France_F1,0.474162,0.025838,France_F2,0.474434,0.025566,Germany_D1,0.471964,0.028036,Finland_Veikkausliiga,0.471111,0.028889
6,Germany_D1,0.475424,0.024576,Germany_D1,0.477350,0.022650,France_F1,0.474021,0.025979,Poland_Ekstraklasa,0.476048,0.023952
7,Belgium_B1,0.477150,0.022850,Poland_Ekstraklasa,0.477564,0.022436,Belgium_B1,0.475546,0.024454,Brazil_Serie A,0.520730,0.020730
8,Poland_Ekstraklasa,0.477564,0.022436,Austria_Bundesliga,0.478151,0.021849,Poland_Ekstraklasa,0.477564,0.022436,France_F2,0.481265,0.018735
9,Austria_Bundesliga,0.478151,0.021849,Argentina_Primera Division,0.478163,0.021837,Austria_Bundesliga,0.478151,0.021849,Spain_SP2,0.484686,0.015314


In [174]:
candidates = []

for col in df_candidates.columns:
    if 'Coun' in col:
        candidates += df_candidates[col].tolist()

candidates = pd.Series(candidates)
candidates.value_counts().head(20)
candidates = candidates.value_counts().head(20).index.tolist()
candidates

['Japan_J-League',
 'Italy_I2',
 'France_F2',
 'Austria_Bundesliga',
 'Finland_Veikkausliiga',
 'Poland_Ekstraklasa',
 'Netherlands_N1',
 'Germany_D1',
 'Germany_D2',
 'Belgium_B1',
 'France_F1',
 'Italy_I1',
 'Argentina_Primera Division',
 'Switzerland_Super League',
 'Scotland_SC1',
 'Spain_SP2',
 'Brazil_Serie A',
 'Scotland_SC0',
 'Ireland_Premier Division',
 'Greece_G1']

# Global

In [180]:
df_res_all = None

for start_year in ['2000', '2005', '2010', '2015', '2019']:
    df_all = df_all_raw[df_all_raw['year'] >= start_year]
    df_all = df_all[df_all['Country_Div'].isin(candidates)]
    df_all_cnt = df_all.groupby(['Country_Div']).count().reset_index()
    df_all_cnt.rename(columns={'year': 'Count'}, inplace=True)
    df_all_cnt = df_all_cnt[df_all_cnt['Count'] > 100]
    df_all_mean = df_all.groupby(['Country_Div']).mean().reset_index()
    df_res = pd.merge(df_all_mean, df_all_cnt[['Country_Div']])
    df_res['chance'] = (0.5 - df_res['total_goals_odd']).abs()
    df_res = df_res.sort_values('chance', ascending=False).reset_index(drop=True)
    df_res['chance_rank'] = range(1, len(df_res)+1, 1)
    df_res.rename(columns={'total_goals_odd': 'odd{}'.format(start_year),
#                           'Count': 'Count_{}'.format(start_year),
                          'chance': 'chance{}'.format(start_year),
                          'chance_rank':'rank{}'.format(start_year)}, inplace=True)
    
    if df_res_all is None:
        df_res_all = df_res
    else:
        df_res_all = pd.merge(df_res_all, df_res)
    

In [181]:
df_res_all.head()

,Country_Div,odd2000,chance2000,rank2000,odd2005,chance2005,rank2005,odd2010,chance2010,rank2010,odd2015,chance2015,rank2015,odd2019,chance2019,rank2019
0,Japan_J-League,0.539749,0.039749,1,0.539749,0.039749,1,0.539749,0.039749,2,0.540636,0.040636,1,0.571429,0.071429,4
1,Netherlands_N1,0.466103,0.033897,2,0.465434,0.034566,3,0.457516,0.042484,1,0.461874,0.038126,2,0.483660,0.016340,11
2,Germany_D2,0.467286,0.032714,3,0.464747,0.035253,2,0.463649,0.036351,3,0.463203,0.036797,4,0.425926,0.074074,2
3,France_F2,0.469457,0.030543,4,0.474434,0.025566,6,0.470344,0.029656,4,0.481265,0.018735,9,0.500000,0.000000,16
4,Italy_I2,0.471715,0.028285,5,0.472649,0.027351,5,0.471462,0.028538,5,0.469500,0.030500,5,0.493421,0.006579,15


In [182]:
df_res_all['rank'] = (df_res_all['rank2000'] + df_res_all['rank2005'] +
                                  df_res_all['rank2010'] + df_res_all['rank2015'] + 
                                  df_res_all['rank2019']) / 5
df_res_all.sort_values('rank', inplace=True)
df_res_all.reset_index(inplace=True, drop=True)

In [183]:
show_cols = ['Country_Div', 'odd2000',
       'rank2000', 'odd2005', 
       'rank2005', 'odd2010', 
       'rank2010', 'odd2015', 
       'rank2015', 'odd2019', 
       'rank2019', 'rank']

In [184]:
df_res_all[show_cols]

,Country_Div,odd2000,rank2000,odd2005,rank2005,odd2010,rank2010,odd2015,rank2015,odd2019,rank2019,rank
0,Japan_J-League,0.539749,1,0.539749,1,0.539749,2,0.540636,1,0.571429,4,1.8
1,Germany_D2,0.467286,3,0.464747,2,0.463649,3,0.463203,4,0.425926,2,2.8
2,Netherlands_N1,0.466103,2,0.465434,3,0.457516,1,0.461874,2,0.483660,11,3.8
3,Germany_D1,0.475424,7,0.477350,7,0.471964,6,0.461874,3,0.437908,5,5.6
4,Italy_I2,0.471715,5,0.472649,5,0.471462,5,0.469500,5,0.493421,15,7.0
5,Poland_Ekstraklasa,0.477564,9,0.477564,8,0.477564,9,0.476048,7,0.427632,3,7.2
6,France_F2,0.469457,4,0.474434,6,0.470344,4,0.481265,9,0.500000,16,7.8
7,France_F1,0.474162,6,0.467721,4,0.474021,7,0.496763,19,0.469274,10,9.2
8,Belgium_B1,0.477150,8,0.482590,13,0.475546,8,0.491964,16,0.425000,1,9.2
9,Austria_Bundesliga,0.478151,10,0.478151,9,0.478151,10,0.486747,13,0.462963,9,10.2


# Kelly formula

In [193]:
p = 0.53
b = 0.94
q = 1 - p
fare = (b*p - q) / b
fare

0.030000000000000006

In [94]:
df_all_year_mean = df_all.groupby(['Country_Div', 'year']).mean()

df_all_year_mean.reset_index(level='year', inplace=True)

for cd in  df_all_year_mean.index.unique():
    df_plot = df_all_year_mean.loc[cd]
    df_plot['total_goals_odd'] -= 0.5
    df_plot.set_index('year', inplace=True)
    df_plot.plot.bar(title=cd, legend=False)